## Init TF

In [1]:
from utils import *
import numpy as np
import tensorflow as tf
# from tensorflow.python.framework.ops import disable_eager_execution
import time

#need to have these two lines to work on my ancient 1060 3gb
#  https://stackoverflow.com/questions/43990046/tensorflow-blas-gemm-launch-failed
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

#turn off eager execution
# disable_eager_execution() #no?

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2
%autosave 180
# %matplotlib notebook

Autosaving every 180 seconds


## Inline pointcloud rendering with Vedo

In [100]:
## from vedo import *
from utils import *
import numpy as np
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti

settings.embedWindow(backend='ipyvtk', verbose = True) #was this

try:
    plt1.closeWindow()
    print("closed")
except:
    pass

plt1 = Plotter(N=1, axes=1, bg = (0.1,0.1,0.1), bg2 = (0.3,0.3,0.3),  interactive=True)
# settings.useParallelProjection = True #makes view orthographic

## uncomment to use VOLPE dataset -----------------------------------------------------
# location = 'C:/Users/Derm/2021-03-10-16-43-50_Velodyne-VLP-16-Data_garminSignage.txt'
# cloud = np.loadtxt(open(location, "rb"), delimiter=",")
# cloud = cloud[~np.isnan(cloud).any(axis=1)] #remove all rows with NaN elements
## ------------------------------------------------------------------------------------

## uncomment to use KITTI dataset -----------------------------------------------------
basedir = 'C:/kitti/'
date = '2011_09_26'
drive = '0005'
frame_range = range(150, 151, 1)
dataset = pykitti.raw(basedir, date, drive)
velo1 = dataset.get_velo(0) # Each scan is a Nx4 array of [x,y,z,reflectance]
cloud = velo1[:,:3]
print(type(cloud))
## ------------------------------------------------------------------------------------

# make 2D sinusioal motion (for debug) ------------------------------------------------
# cloud = np.random.randn(10000,3)
# cloud[:,0] += -50*cloud[:,1] + np.random.randn()*5
# cloud[:,1] += 5
# cloud[:,1] = cloud[:,1] * 3 + 10*np.random.randn() + np.sin(cloud[:,1]*5)*10
# cloud[:,2] += 0.5*cloud[:,1] - 10
##-------------------------------------------------------------------------------------

# f =np.array([200,200,40]) #fidelity in x, y, z #takes ~30s on my dsektop
# lim = np.array([-50,50,-50,50,-10,10])

f =np.array([50,50,1]) #fidelity in x, y, z # < 5s
lim = np.array([-100,100,-100,100,-10,10])

E = subdivide_scan(cloud,plt1, bounds = lim, fid = f, draw_grid = False, show_pc = False) 
ViewInteractiveWidget(plt1.window)

closed
<class 'numpy.ndarray'>


ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)

## Unit Test R() and Jacobian

In [101]:
n_hat = np.array([1,0,0])
theta =  0.1 #np.pi/6 #rad

rot_mat_simp = R_simp(n_hat, theta)
print(rot_mat_simp)
angs = np.array([theta,0 ,0])
rot_mat = R(angs)
print(rot_mat)

print(R2Euler(rot_mat))

p_point = np.array([1,1,1]).T

J = jacobian(angs, p_point)

d_rot_mat_simp = dR_simp(n_hat,theta)
# print(d_rot_mat_simp)

[[ 1.          0.          0.        ]
 [ 0.          0.9950042  -0.09983342]
 [ 0.          0.09983342  0.9950042 ]]
[[ 1.          0.          0.        ]
 [-0.          0.99500417  0.09983342]
 [ 0.         -0.09983342  0.99500417]]
[-0.1  0.  -0. ]



## Inline volumetric rendering using ipyvolume

In [ ]:
#simple demo
import ipyvolume
ds = ipyvolume.datasets.aquariusA2.fetch()
short = ds.data[:,:,:]
ipyvolume.quickvolshow(short, lighting=True)

In [102]:
ans = np.linspace(1,12,8)
ans[2] = 0
print(ans)

test = ans[ans < 10]
print(test)
np.shape(test)[0]

print(np.median(test[:3]))

[ 1.          2.57142857  0.          5.71428571  7.28571429  8.85714286
 10.42857143 12.        ]
[1.         2.57142857 0.         5.71428571 7.28571429 8.85714286]
1.0


tf.Tensor([10.], shape=(1,), dtype=float32)
tf.Tensor([ 0.84147098  0.53977018  0.         -0.53870529  0.84283471  0.53763953], shape=(6,), dtype=float64)


<tf.Tensor: shape=(), dtype=float32, numpy=-0.54402107>

In [ ]:
# a = tf.reshape([10.,10.,2.], (3,1))
# print(type(a))
# t = tf.Tensor(a, dtype = "float32")

a = tf.constant([2.1,2.,3.])
b = tf.constant([1.,2.,3.])
tf.tensordot(a,b, axes = 0)

In [ ]:
eye = tf.eye(3)
# print(j)
Jx = tf.constant([[1.], [2.], [3.]])
J = tf.concat([eye, Jx], axis = 1)
print(J)

In [95]:
# print(tf.sin(1.))
start = time.time()
numiter = 100
angs  = np.array([1.,0.1,0.1])
p_point = np.array([1.,2.,3.])
for _ in range(numiter):
    J = jacobian(angs, p_point);
print("took", time.time()-start, "seconds on CPU")
print(J)

took 0.4831082820892334 seconds on CPU
[[ 1.          0.          0.          0.35191004 -0.0379179   3.48705454]
 [ 0.          1.          0.         -0.0976553   0.00380448 -1.35609674]
 [ 0.          0.          1.         -3.58700749  1.00119734  0.        ]]


In [98]:
#TF is slower if we do them one by one BUT is waaay faster if we send them in all at once
start = time.time()
numiter = 100000
angs = tf.random.normal((3,numiter))
p_point = tf.random.normal((3,numiter))
J = jacobian_tf(angs, p_point);
print("took", time.time()-start, "seconds on GPU")
print(J.numpy())

took 0.011002779006958008 seconds on GPU
[[ 1.          0.          0.         -1.0232804  -0.2863361   0.9867529 ]
 [ 0.          1.          0.          0.32353926 -0.42546123 -0.17956361]
 [ 0.          0.          1.         -0.26423293  0.9188831   0.        ]
 ...
 [ 1.          0.          0.         -0.03568125  0.03095145  0.08746541]
 [ 0.          1.          0.          0.39335006  0.04599016 -0.54896903]
 [ 0.          0.          1.          0.26727706  0.37907043  0.        ]]


## Unit test R2Euler_tf and R_tf
#### Works with single axis roation
#### Works with vectoried input
#### Solution becomes ambiguious with mutliple axis inputs

In [104]:
angs = tf.random.normal((3,1)) * tf.constant([[0.], [0.], [1.] ]) #single axis angle input
# angs = tf.Variable([[0.],[ np.pi/2],[ 0.]]) #vector input, multiple axis rotation
print("Input angs: \n", angs.numpy())
# print("R(angs): \n", R_tf(angs).numpy())
test1 = R2Euler_tf(R_tf(angs))
print("R2Euler_tf(R(angs)): \n", test1.numpy())
# test2 = R2Euler(R(angs[:,:2]))
# print(test2)

Input angs: 
 [[ 0.       ]
 [-0.       ]
 [ 1.3544165]]
R2Euler_tf(R(angs)): 
 [[-0.       ]
 [ 0.       ]
 [-1.3544165]]
